In [1]:
import pandas as pd

In [2]:
# Create two dataframes
df = pd.DataFrame({'BOOKING_NO': [1, 2, 3, 4], 'COL_A': ['A', 'B', 'C', 'D']})
df2 = pd.DataFrame({'BOOKING_NO': [1, 2, 2, 3, 3, 3], 'COL_B': ['X', 'Y', 'Z', 'W', 'X', 'Y']})

In [23]:
bookings2 = """503200025365
503200025390
503200025390
503200025403
503200025403
503200025403
503200025403
50320005403
503200025403
503200025403
503200025403
503200025403
503200025403
503200025705
503200025705
503200025705
503200025705
503200025705
503200025845""".split()

bookings = """503200025365
503200025390
503200025390
503200025403
503200025403
503200025403
503200025403
503200025403
503200025403
503200025403
503200025403
503200025403
503200025403
503200025705
503200025705
503200025705
503200025705
503200025705
503200025705
503200025705
503200025705
503200025845
""".split()

containers = """DFSU6896599
EISU9363352
TGBU6778406
EGHU1003711
EGHU1019723
EGHU1066833
EISU1737815
EISU1851828
HMCU1097488
HMCU1106698
LTIU6026655
LTIU6051241
TCLU4342545
DRYU9308105
EGHU8478600
EISU9482198
EMCU8909929
TGBU7137602
TLLU4025385
""".split()

nwt = """9990
8344
14692
14040
11520
12850
12720
6472
6785
14170
14740
6404
13580
20464
25278
25374
25969
24699
24765
25619
20683
21185
""".split()
nwt2 = """13.79
12.22
18.39
17.67
15.15
16.47
16.35
10.19
10.50
17.88
18.68
29.79
28.37
24.89
24.70
31.00
32.50
29.80
31.85
""".split()


In [24]:
df = pd.DataFrame({"BOOKING_NO": bookings, 'NET_WEIGHT': nwt})
df2 = pd.DataFrame({"BOOKING_NO": bookings2, 'CONTAINER': containers, 'NET_WEIGHT': nwt2})

ValueError: All arrays must be of the same length

In [25]:
merged_df = pd.merge(df, df2, on='BOOKING_NO', how='left')

In [22]:
merged_df.to_excel('test.xlsx')

In [ ]:
# Merge the two dataframes based on the 'BOOKING_NO' column
merged_df = pd.merge(df, df2, on='BOOKING_NO', how='left')
print(merged_df)

In [ ]:
# Group by 'BOOKING_NO' and count the number of instances in DF2
grouped = df2.groupby('BOOKING_NO').size().reset_index(name='counts')

In [ ]:
print(grouped)

In [ ]:
for index, row in grouped.iterrows():
    print(row['counts'])

In [ ]:
# For each 'BOOKING_NO' with more than one instance, add rows to the merged dataframe
for index, row in grouped.iterrows():
    if row['counts'] > 1:
        for i in range(row['counts'] - 1):
            booking_no = row['BOOKING_NO']
            data = df2.loc[df2['BOOKING_NO'] == booking_no].iloc[i+1]
            merged_df = merged_df.append({'BOOKING_NO': booking_no, 'COL_A': data['COL_B']}, ignore_index=True)
            

In [ ]:
print(merged_df)